In [6]:
import pandas as pd
import backtrader as bt
import datetime
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class Ativo:
    def __init__(self, ticker, setor, ohlc):
        self.ticker = str(ticker)
        self.setor = str(setor)
        self.ohlc = list(ohlc)
    pass

In [3]:
ativo1 = Ativo('PETR4', 'Combustíveis', [32.35, 33.17, 31.55, 31.93])
ativo1.ticker

'PETR4'

In [4]:
ativo1.ohlc[3]

31.93

In [5]:
ativo2 = Ativo('ITUB4', 'Financeiro', [25.32, 27.01, 25.18, 26.77])
ativo3 = Ativo('VALE3', 'Mineração', [80.87, 82.55, 80.76, 82.40])

In [7]:
class Ativo:
    def __init__(self, ticker, setor, ohlc):
        self.ticker = str(ticker)
        self.setor = str(setor)
        self.ohlc = list(ohlc)
    
    def Amplitude(self):
        print(round((self.ohlc[1] - self.ohlc[2]), 2))

    def Tipo(self):
        if self.ticker[-1] == '3':
            print(self.ticker, 'é uma ação do tipo Ordinária')
        elif self.ticker[-1] == '4':
            print(self.ticker, 'é uma ação do tipo Preferencial')
        elif self.ticker[-1] == '1':
            print(self.ticker, 'é uma ação do tipo UNIT')
        else:
            print('Não tenho certeza, melhor consultar o site da B3')

    def Resultado(self):
        resultado = self.ohlc[3] - self.ohlc[0]
        if resultado == 0:
            print(self.ticker, 'fechou estável')
        elif resultado > 0:
            print(self.ticker, 'fechou em alta')
        else:
            print(self.ticker, 'fechou em baixa')

    def Fech_prox_max_min(self):
        media_max_min = np.mean([self.ohlc[1], self.ohlc[2]])
        if self.ohlc[3] > media_max_min:
            print(self.ticker, 'parece estar em tendência de alta')
        else:
            print(self.ticker, 'parece estar em tendência de baixa')

    pass


In [8]:
ativo1 = Ativo('PETR4', 'Combustíveis', [32.35, 33.17, 31.55, 31.93])
ativo2 = Ativo('ITUB4', 'Financeiro', [25.32, 27.01, 25.18, 26.77])
ativo3 = Ativo('VALE3', 'Mineração', [80.87, 82.55, 80.76, 82.40])
ativo4 = Ativo('BPAC11', 'Financeiro', [24,17, 24.77, 23.70, 23.89])

In [9]:
ativo1.Amplitude()

1.62


In [10]:
Ativo.Amplitude(ativo3)

1.79


In [11]:
ativo4.Tipo()

BPAC11 é uma ação do tipo UNIT


In [12]:
ativo2.setor == ativo4.setor

True

In [13]:
Ativo.Fech_prox_max_min(ativo3)

VALE3 parece estar em tendência de alta


# Capturando dados do Yahoo Finance

In [15]:
data = bt.feeds.PandasData(dataname=yf.download('PETR4.SA', start='2017-01-01', end='2021-07-01', auto_adjust=True))

[*********************100%***********************]  1 of 1 completed


# Estruturadas de estratégia dentro do Backtrader

In [16]:
class PrintFechamento(bt.Strategy):
    def __init__(self):
        self.dataclose = self.datas[0].close # deixar como referência a linha final [0] da séria de dados
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sellprice = None
        self.sellcomm = None
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s - %s' % (dt.isoformat(), txt))

    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])

In [18]:
# Iniciar a engine
cerebro = bt.Cerebro()

# Adicionar os dados para o backtest
cerebro.adddata(data)

# Adicionar a estratégia
cerebro.addstrategy(PrintFechamento)

# Executar
cerebro.run()

2017-01-02 - Close, 9.07
2017-01-03 - Close, 9.59
2017-01-04 - Close, 9.59
2017-01-05 - Close, 9.74
2017-01-06 - Close, 9.69
2017-01-09 - Close, 9.48
2017-01-10 - Close, 9.57
2017-01-11 - Close, 9.69
2017-01-12 - Close, 9.83
2017-01-13 - Close, 9.70
2017-01-16 - Close, 9.74
2017-01-17 - Close, 9.78
2017-01-18 - Close, 9.77
2017-01-19 - Close, 9.75
2017-01-20 - Close, 9.90
2017-01-23 - Close, 9.89
2017-01-24 - Close, 9.92
2017-01-26 - Close, 9.77
2017-01-27 - Close, 9.66
2017-01-30 - Close, 9.18
2017-01-31 - Close, 9.29
2017-02-01 - Close, 9.29
2017-02-02 - Close, 9.21
2017-02-03 - Close, 9.49
2017-02-06 - Close, 9.25
2017-02-07 - Close, 9.09
2017-02-08 - Close, 9.34
2017-02-09 - Close, 9.31
2017-02-10 - Close, 9.64
2017-02-13 - Close, 9.66
2017-02-14 - Close, 9.78
2017-02-15 - Close, 9.80
2017-02-16 - Close, 9.81
2017-02-17 - Close, 9.65
2017-02-20 - Close, 9.85
2017-02-21 - Close, 9.95
2017-02-22 - Close, 9.71
2017-02-23 - Close, 9.62
2017-02-24 - Close, 9.39
2017-03-01 - Close, 9.60


# Estratégia Básica
Cruzamento de média móvel simples

In [19]:
wege3 = yf.download('WEGE3.SA', start='2011-01-01', end='2021-07-01', auto_adjust=True, interval='5d')
wege3.dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed


In [20]:
data2 = bt.feeds.PandasData(dataname=wege3)

In [21]:
from backtrader.indicators import crossover

# Para rodar melhor o matplotplib
%matplotlib inline

class MMACruzamento(bt.SignalStrategy):
    def __init__(self):
        sma = bt.ind.SMA(period=26)
        price = self.data
        crossover = bt.ind.CrossOver(price, sma)
        self.signal_add(bt.SIGNAL_LONG, crossover)

cerebro = bt.Cerebro()
cerebro.addstrategy(MMACruzamento)
cerebro.adddata(data2)
cerebro.run()

plt.rcParams['figure.figsize'] = (15, 12)
plt.rcParams.update({'font.size': 12})
cerebro.plot(iplot=False)

ImportError: cannot import name 'warnings' from 'matplotlib.dates' (c:\Users\mbalt\miniconda3\envs\curso_modal\lib\site-packages\matplotlib\dates.py)

Se quiser rodar este código no jupyter tem que fazer o workaround explicado no link abaixo, devido a uma incompatibilidade entre backtrader e matplotlib:

>André L V Coelho
>
>24 dias atrás
>
>Olá. Consegui executar o notebook no jupyterlab. Tive problema para plotar os gráficos, pois há um problema de compatibilidade entre as bibliotecas backtrader e matplotlib. Consegui resolvê-lo alterando o arquivo "backtrader/plot/locator.py", seguindo o que está aqui:

https://community.backtrader.com/topic/981/importerror-cannot-import-name-min_per_hour-when-trying-to-plot/8
